In [1]:
import sys
sys.path.append(r"C:\Users\Trace\OneDrive\Desktop\Crypto_Multi_Venue_StatArb\altcoin_pairs")

import importlib
import utilities.loader
importlib.reload(utilities.loader)

from utilities.loader import load_binance_for_backtester
from utilities.sectors import SECTOR_MAPPING
import pandas as pd

In [2]:
all_sectors = {"all": [item for items in SECTOR_MAPPING.values() for item in items]}

In [3]:
data = load_binance_for_backtester("./data_collect/data_complete")

Found 405 symbols
Loaded 405 symbols for venue 'binance'


In [4]:
from utilities.backtest_engine import run_pairs, print_trade_log

results = run_pairs(
    data,
    sectors=all_sectors,
    venue="binance",
    initial_capital = 1_000_000,

    # these two must be consistent -- screen_lookback <= formation_window
    formation_window=6000,    # how much history the Kalman warms up on
    screen_lookback=3000,     # how much the EG test looks at (shorter is fine)
    
    refit_interval=720,       # re-screen every ~30 days at 96 bars/day

    # screening
    max_p=0.001,                 
        
    entry_z=2.0,        
    exit_z=0.75,
    # signals
    z_hard_stop=6, 
    fdr_q = 0.001,

    rho_high=0.8, 
    rho_low=0.10,

    # risk
    position_pct=0.05,
    pct_stop=0.10,
    
    max_coin_appearances = 3,
    
    stale_z_progress=-999,
    hl_max_mult = 0,

    max_sector_pairs=30,
    max_pairs= 30,
    max_open = 30,
    
    confidence_power = 3.5,

    hl_optimal_low = 12.0,
    hl_optimal_high = 64.0,

    max_leverage = 3.0,

    output_dir="./reports",
    btc_symbol="BTC",
)

print_trade_log(results.trade_log, n=20)

[feed] 405 coins (32 full, 373 partial), 140251 bars, log_prices=ON

  Running: 140251 bars, 405 coins | venue=binance | capital=$1,000,000 | leverage=3.0x
    screening: 49/16907 pairs passed (BH q=0.001, KPSS+VR<1.0+|skew|<1.0) in 5.6s
    ranking: 26/49 pairs selected (score 0.698→0.576, 1 sectors, 20 coins)
  [2022-02-01] REFIT: 26 pairs from 98/405 coins | 6.6s
    screening: 70/17170 pairs passed (BH q=0.001, KPSS+VR<1.0+|skew|<1.0) in 1.4s
    ranking: 30/70 pairs selected (score 0.720→0.535, 1 sectors, 23 coins)
  [2022-02-08] REFIT: 30 pairs from 98/405 coins | 1.5s
    screening: 112/17448 pairs passed (BH q=0.001, KPSS+VR<1.0+|skew|<1.0) in 1.4s
    ranking: 30/112 pairs selected (score 0.689→0.620, 1 sectors, 22 coins)
  [2022-02-16] REFIT: 30 pairs from 98/405 coins | 1.6s
    screening: 79/17586 pairs passed (BH q=0.001, KPSS+VR<1.0+|skew|<1.0) in 1.4s
    ranking: 30/79 pairs selected (score 0.676→0.585, 1 sectors, 27 coins)
  [2022-02-23] REFIT: 30 pairs from 98/405 coi

In [5]:
# Check actual leverage throughout the backtest
eq_df = results.equity_curve
if 'gross_exposure' in eq_df.columns:
    leverage = eq_df['gross_exposure'] / eq_df['equity']
    print(f"Mean leverage:   {leverage.mean():.2f}x")
    print(f"Max leverage:    {leverage.max():.2f}x")
    print(f"Median leverage: {leverage.median():.2f}x")
else:
    # estimate from trade log
    tl = results.trade_log
    print(f"Mean notional per trade: ${tl['notional'].mean():,.0f}")
    print(f"Median open positions:   check equity snapshots")

Mean notional per trade: $168,307
Median open positions:   check equity snapshots


In [6]:
import matplotlib.pyplot as plt
# Crisis period analysis
crisis_periods = {                                                                                                                                                                                               
    "UST/Luna Collapse": (pd.Timestamp("2022-05-01", tz="UTC"), pd.Timestamp("2022-06-15", tz="UTC")),
    "FTX Bankruptcy": (pd.Timestamp("2022-11-01", tz="UTC"), pd.Timestamp("2022-12-15", tz="UTC")),                                                                                                              
    "USDC Depeg / Bank Crisis": (pd.Timestamp("2023-03-01", tz="UTC"), pd.Timestamp("2023-04-01", tz="UTC")),
    "SEC Lawsuits (Binance/Coinbase)": (pd.Timestamp("2023-06-01", tz="UTC"), pd.Timestamp("2023-07-15", tz="UTC")),                                                                                             
}

df_trades =  results.trade_log

# map entry_bar to timestamp from equity curve
eq_df = results.equity_curve
bar_to_ts = dict(zip(eq_df["bar"], pd.to_datetime(eq_df["timestamp"])))
df_trades["entry_ts"] = df_trades["entry_bar"].map(bar_to_ts)
df_trades["exit_ts"] = df_trades["exit_bar"].map(bar_to_ts)

# per-crisis stats
crisis_stats = []
for name, (start_ts, end_ts) in crisis_periods.items():
    mask = (
        (df_trades["entry_ts"] <= end_ts) &
        (df_trades["exit_ts"] >= start_ts)
    )
    ct = df_trades[mask]

    if len(ct) == 0:
        crisis_stats.append({"Crisis": name, "Trades": 0})
        continue

    wins = ct[ct["net_pnl"] > 0]

    crisis_stats.append({
        "Crisis": name,
        "Period": f"{start_ts.date()} → {end_ts.date()}",
        "Trades": len(ct),
        "Win Rate": f"{len(wins)/len(ct):.1%}",
        "Total P/L": f"${ct['net_pnl'].sum():,.2f}",
        "Avg P/L %": f"{ct['pnl_pct'].mean()*100:.3f}%",
        "Avg P/L $": f"${ct['net_pnl'].mean():,.2f}",
        "Best Trade": f"${ct['net_pnl'].max():,.2f}",
        "Worst Trade": f"${ct['net_pnl'].min():,.2f}",
        "Total Costs": f"${ct['total_cost'].sum():,.2f}",
        "Avg Bars Held": f"{ct['bars_held'].mean():.0f}",
        "Exit Reasons": dict(ct["exit_reason"].value_counts()),
    })

crisis_df = pd.DataFrame(crisis_stats)
print("\n=== CRISIS PERIOD PERFORMANCE ===\n")
for _, row in crisis_df.iterrows():
    print(f"  {row['Crisis']}")
    if row["Trades"] == 0:
        print("    No trades during this period\n")
        continue
    print(f"    Period:       {row['Period']}")
    print(f"    Trades:       {row['Trades']}  (WR: {row['Win Rate']})")
    print(f"    Total P/L:    {row['Total P/L']}  |  Avg: {row['Avg P/L $']}  ({row['Avg P/L %']})")
    print(f"    Best/Worst:   {row['Best Trade']} / {row['Worst Trade']}")
    print(f"    Costs:        {row['Total Costs']}")
    print(f"    Avg Hold:     {row['Avg Bars Held']} bars")
    print(f"    Exits:        {row['Exit Reasons']}")
    print()

# chart: P/L by crisis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

names = [s["Crisis"] for s in crisis_stats if s["Trades"] > 0]
totals = [df_trades[
    (df_trades["entry_ts"] <= crisis_periods[n][1]) &
    (df_trades["exit_ts"] >= crisis_periods[n][0])
]["net_pnl"].sum() for n in names]
colors = ["#2ca02c" if v >= 0 else "#d62728" for v in totals]

axes[0].barh(names, totals, color=colors, alpha=0.7)
axes[0].axvline(0, color="black", linestyle="--", alpha=0.4)
axes[0].set_xlabel("Total P/L ($)")
axes[0].set_title("Total P/L by Crisis")

win_rates = [df_trades[
    (df_trades["entry_ts"] <= crisis_periods[n][1]) &
    (df_trades["exit_ts"] >= crisis_periods[n][0])
]["net_pnl"].apply(lambda x: x > 0).mean() for n in names]

axes[1].barh(names, win_rates, color="#1f77b4", alpha=0.7)
axes[1].axvline(0.5, color="black", linestyle="--", alpha=0.4)
axes[1].set_xlabel("Win Rate")
axes[1].set_title("Win Rate by Crisis")
axes[1].set_xlim(0, 1)

plt.tight_layout()
plt.show()

# sector breakdown per crisis
print("\n=== SECTOR BREAKDOWN PER CRISIS ===\n")
for name, (start_ts, end_ts) in crisis_periods.items():
    mask = (df_trades["entry_ts"] <= end_ts) & (df_trades["exit_ts"] >= start_ts)
    ct = df_trades[mask]
    if len(ct) == 0:
        continue
    print(f"  {name}:")
    sector_pnl = ct.groupby("sector")["net_pnl"].agg(["sum", "count", "mean"])
    sector_pnl.columns = ["Total P/L", "Trades", "Avg P/L"]
    sector_pnl = sector_pnl.sort_values("Total P/L", ascending=False)
    with pd.option_context("display.float_format", "{:.2f}".format):
        print(sector_pnl.to_string(index=True))
    print()


=== CRISIS PERIOD PERFORMANCE ===

  UST/Luna Collapse
    Period:       2022-05-01 → 2022-06-15
    Trades:       383  (WR: 67.6%)
    Total P/L:    $169,088.96  |  Avg: $441.49  (0.491%)
    Best/Worst:   $25,230.98 / $-22,402.69
    Costs:        $67,375.83
    Avg Hold:     26 bars
    Exits:        {'signal': np.int64(350), 'forced': np.int64(17), 'stop_loss': np.int64(9), 'z_stop': np.int64(4), 'beta_break': np.int64(3)}

  FTX Bankruptcy
    Period:       2022-11-01 → 2022-12-15
    Trades:       186  (WR: 68.8%)
    Total P/L:    $87,805.92  |  Avg: $472.07  (0.264%)
    Best/Worst:   $9,398.13 / $-20,867.97
    Costs:        $34,241.91
    Avg Hold:     25 bars
    Exits:        {'signal': np.int64(165), 'forced': np.int64(15), 'stop_loss': np.int64(3), 'z_stop': np.int64(2), 'beta_break': np.int64(1)}

  USDC Depeg / Bank Crisis
    Period:       2023-03-01 → 2023-04-01
    Trades:       134  (WR: 64.9%)
    Total P/L:    $71,615.29  |  Avg: $534.44  (0.134%)
    Best/Worst:

In [7]:
initial_capital = 1_000_000
total_net_pnl = df_trades["net_pnl"].sum()
total_costs = df_trades["total_cost"].sum()
ending_equity = initial_capital + total_net_pnl

print(f"Initial Capital:  ${initial_capital:,.2f}")
print(f"Total Net P/L:    ${total_net_pnl:,.2f}")
print(f"Total Costs:      ${total_costs:,.2f}")
print(f"Ending Equity:    ${ending_equity:,.2f}")
print(f"Total Return:     {(ending_equity/initial_capital - 1)*100:.2f}%")

Initial Capital:  $1,000,000.00
Total Net P/L:    $1,459,279.88
Total Costs:      $1,554,736.01
Ending Equity:    $2,459,279.88
Total Return:     145.93%


In [8]:
import numpy as np
import pandas as pd

eq = results.equity_curve.copy()
eq["timestamp"] = pd.to_datetime(eq["timestamp"])
daily_eq = eq.set_index("timestamp")["total_equity"].resample("1D").last().dropna()

daily_ret = daily_eq.pct_change().dropna()
excess = daily_ret - 0.04 / 365

sharpe = np.mean(excess) / np.std(excess) * np.sqrt(365)
print(f"Daily Sharpe: {sharpe:.3f}")

Daily Sharpe: 1.439
